In [7]:
import urllib.parse
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
import pandas as pd

In [8]:
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()
count = 0

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)


In [ ]:
#test blank nodes
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()
count = 0

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)
bn = BNode()


g.add([URIRef(base_uri + "2"), sso.hasSites, bn])
g.add([bn, sso.hasCap, Literal(90)])
g.add([bn, sso.hasAddress, Literal("SHJSHJ")])

bn2 = BNode()
g.add([URIRef(base_uri + "2"), sso.hasSites, bn2])
g.add([bn2, sso.hasCap, Literal(22)])
g.add([bn2, sso.hasAddress, Literal("aaa")])

g.serialize(destination='../datasets/rdf/bndoes.ttl', 
            format='turtle')

In [9]:
def urify(ns, testo):
    testo=testo.replace(" ","_").replace("\'","")
    return ns+urllib.parse.quote(testo)

In [ ]:
import re

#Comuni
def addTriples(row):
    res = URIRef(urify(base_uri, row[0]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Comune)])
    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string)])
    g.add([res, sso.hasTotalPopulation, Literal(row[1], datatype=XSD.integer)])
    g.add([res, sso.hasProvince, Literal(row[2], datatype=XSD.string)])
    g.add([res, sso.hasProvinceAcr, Literal(row[3], datatype=XSD.string)])
    #interlinking
    a = row[0].replace('a\'', 'à').replace('o\'', 'ò').replace('e\'', 'è').replace('i\'', 'ì').replace('u\'', 'ù')
    g.add([res, RDFS.seeAlso, URIRef(urify("https://www.dbpedia.org/resource/", a))])


comuni_df = pd.read_csv("../datasets/csv/completed/comuni_sicilia.csv")

comuni_df.apply(lambda row : addTriples(row), axis=1)


In [10]:
#Farmacie
count = 0
def addTriples(row):
    global count
    code = 'F'+f'{count:04}'
    #sostituire con codice farmacia/parafarmacia piuttosto che nome 
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.Farmacia)])
    g.add([res, sso.isIn, URIRef(urify(base_uri, row[2].title()))])

    if(row[3] != 0):
        g.add([res, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
        
    g.add([res, sso.hasAddress, Literal(row[4], datatype=XSD.string) ])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])

    if(row[6] != 0 and row[7] != 0):
        g.add([res, sso.hasLatitude, Literal(row[6], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[7], datatype=XSD.decimal) ])
    g.add([res, sso.hasVatNumber, Literal(row[8], datatype=XSD.integer) ])
    count = count+1


farmacie_df = pd.read_csv("../datasets/csv/farmacie_new.csv")

farmacie_df.apply(lambda row : addTriples(row), axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/csv/farmacie_new.csv'

In [ ]:
df_parafarmacie = pd.read_csv("../datasets/csv/completed/parafarmacie.csv")
df_parafarmacie["DENOMINAZIONESITOLOGISTICO"].unique().size

In [11]:
#Parafarmacie
count = 0
row_prev = pd.Series()

def addBNodes(res, row):
    bn = BNode()

    g.add([res, sso.hasSite, bn])
    g.add([bn, sso.hasAddress, Literal(row[1], datatype=XSD.string) ])
   
    if(row[3] != 0):
        g.add([bn, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([bn, sso.isIn, URIRef(urify(base_uri, row[4].title()))])

    if(row[7] != 0 and row[8] != 0):
        g.add([bn, sso.hasLatitude, Literal(row[7], datatype=XSD.decimal) ])
        g.add([bn, sso.hasLongitude, Literal(row[8], datatype=XSD.decimal) ])

def addTriples(row):
    global count, row_prev, count_if
    code = 'PF'+f'{count:04}'

    if row[0] == "farmarisparmio":
        print("JKSJSKJ")
    
    if not row_prev.empty:
        if row_prev[0] == row[0]:
            addBNodes(URIRef(urify(base_uri, code)), row)
            return
            
    row_prev = pd.Series(row)

    
    
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.Parafarmacia)])

    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string) ])
    g.add([res, sso.hasVatNumber, Literal(row[2], datatype=XSD.integer) ])

    addBNodes(URIRef(urify(base_uri, code)), row)
    count = count + 1

parafarmacie_df = pd.read_csv("../datasets/csv/completed/parafarmacie.csv")
parafarmacie_df["LATITUDINE"] = parafarmacie_df["LATITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["LONGITUDINE"] = parafarmacie_df["LONGITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["PARTITAIVA"] = parafarmacie_df["PARTITAIVA"].apply(lambda x : x if x != '-' else 0)


parafarmacie_df.apply(lambda row : addTriples(row), axis=1)

            

0      None
1      None
2      None
3      None
4      None
       ... 
752    None
753    None
754    None
755    None
756    None
Length: 757, dtype: object

In [12]:
g.serialize(destination='../datasets/rdf/sanitasicilia.ttl', 
            format='turtle')

<Graph identifier=N959ab07bd03e4ed7af75fcdad486ec73 (<class 'rdflib.graph.Graph'>)>

In [ ]:
len(parafarmacie_df.index)

In [ ]:
res = g.query('''SELECT (COUNT(?parafarmacia) AS ?cname) WHERE {?parafarmacia rdf:type sso:Parafarmacia .} ''')

for i in res:
    print(i)

In [ ]:
#Strutture sanitarie private
count = 0
def addTriples(row):
    global count
    code = 'SPVT'+f'{count:04}'
    #sostituire con codice struttura
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.StrutturaPrivata)])

    if(row[0] != 0):
        g.add([res, sso.hasCap, Literal(row[0], datatype=XSD.integer) ])
        
    if(row[1] != 0 and row[2] != 0):
        g.add([res, sso.hasLatitude, Literal(row[1], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[2], datatype=XSD.decimal) ])
    
    g.add([res, sso.hasReferenceAsp, Literal(row[3], datatype=XSD.string)])

    if(row[5] != ''):
        g.add([res, sso.isIn, URIRef(urify(base_uri, row[5].title()))])

    if(row[6] != 0):
        g.add([res, sso.hasAddress, Literal(row[6], datatype=XSD.string) ])
        
    g.add([res, sso.hasName, Literal(row[7], datatype=XSD.string) ])
    g.add([res, sso.businessType, Literal(row[8], datatype=XSD.string)])
    g.add([res, sso.hasSiteType, Literal(row[9], datatype=XSD.string) ])
    count = count + 1

private_df = pd.read_csv("../datasets/csv/completed/private.csv")
private_df["Citta\'"] = private_df["Citta\'"].astype(str)
private_df.apply(lambda row : addTriples(row), axis=1)


In [ ]:
#Strutture sanitarie pubbliche
import math

count = 0
def addTriples(row):
    global count
    code = 'SPUB'+f'{count:04}'
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.StrutturaPubblica)])

    if(row[0] != ''):
        g.add([res, sso.isIn, URIRef(urify(base_uri, row[0].title()))])

    if(row[1] != 0 and row[2] != 0):
        g.add([res, sso.hasLatitude, Literal(row[1], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[2], datatype=XSD.decimal) ])
   
    g.add([res, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([res, sso.hasAddress, Literal(row[4], datatype=XSD.string)])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])
    g.add([res, sso.businessType, Literal(row[7], datatype=XSD.string)])
    g.add([res, sso.hasSiteType, Literal(row[8], datatype=XSD.string)])

    if row[9] != '':
        g.add([res, sso.isRecoveryStructure, Literal(True, datatype=XSD.boolean)])
        g.add([res, sso.hasRecoveryStructureName, Literal(row[10], datatype=XSD.string)])
    else:
        g.add([res, sso.isRecoveryStructure, Literal(False, datatype=XSD.boolean)])

    g.add([res, sso.workingMonths, Literal(row[10], datatype=XSD.integer)])
    g.add([res, sso.openingDays, Literal(row[12], datatype=XSD.integer)])
    g.add([res, sso.weeklyHours, Literal(row[13], datatype=XSD.integer)])
    g.add([res, sso.hasVatNumber, Literal(row[14], datatype=XSD.integer)])
    g.add([res, sso.hasEmail, Literal(row[15], datatype=XSD.string)])

    if row[16] != '':
        g.add([res, sso.hasWebsite, Literal(row[16], datatype=XSD.string)])
   
    if not math.isnan(row[17]):
        g.add([res, sso.hasPhonePrefix, Literal(row[17], datatype=XSD.integer)])
   
    if not math.isnan(row[18]):
        g.add([res, sso.hasPhoneNumber, Literal(row[18], datatype=XSD.integer)])
    
    if not math.isnan(row[19]):
        g.add([res, sso.hasFaxPrefix, Literal(row[19], datatype=XSD.integer)])
    
    if not math.isnan(row[20]):
        g.add([res, sso.hasFaxNumber, Literal(row[20], datatype=XSD.integer)])
    
    count = count + 1



pubbliche_df = pd.read_csv("../datasets/csv/completed/pubbliche.csv")

pubbliche_df["Comune"] = pubbliche_df["Comune"].astype(str)
pubbliche_df["Denominazione struttura di ricovero"] = pubbliche_df["Denominazione struttura di ricovero"].astype(str)
pubbliche_df["Sito web"] = pubbliche_df["Sito web"].astype(str)

pubbliche_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/rdf/sanitasicilia.ttl', 
            format='turtle')

In [ ]:
res = g.query('''SELECT ?name 
                 WHERE 
                {
                    ?struttura rdf:type sso:StrutturaPrivata .
                    ?struttura sso:hasName ?name .
                    ?struttura sso:isIn ?comune .
                    ?comune sso:hasName ?nomeC .
                    FILTER REGEX (?nomeC, "Palermo") .
                }
 ''')


In [ ]:
for i in res:
    print(i)